In [ ]:
#실행하고 마지막에 input으로 csv이름 넣으면 됨.
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains
import time

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
import pandas as pd

#크롬창 보기 싫으면 주석 해제하셈(webdriver.Chrome 옵션까지 해제해야함)
#webdriver_options = webdriver.ChromeOptions()
#webdriver_options .add_argument('headless')

#url 및 웹드라이버 설정
URL = ''
driver = webdriver.Chrome(executable_path='') #, options = webdriver_options)
driver.get(url = URL)

# 로그인
driver.implicitly_wait(time_to_wait = 5)
driver.find_element_by_id('userId').send_keys('아이디')
driver.find_element_by_id('password').send_keys('비밀번호')
driver.find_element_by_id('loginSubmit').click()

# 기존 사용 중지 후 로그인 알림창 넘어가기
try:
    WebDriverWait(driver, 3).until(EC.alert_is_present())
    alert = driver.switch_to.alert
    alert.accept()
except:
    print("경고창이 없네용")

# Q & A 까지 들어가기
driver.implicitly_wait(time_to_wait = 10)
driver.find_element_by_id('course-link-_71737_1').click()
driver.implicitly_wait(time_to_wait = 5)
driver.get("https://eclass2.ajou.ac.kr/webapps/discussionboard/do/forum?action=list_threads&course_id=_71737_1&forum_id=88237&nav=discussion_board&conf_id=_83802_1&mode=view")
driver.implicitly_wait(time_to_wait = 5)
driver.find_element_by_id("listContainer_row:0")
driver.implicitly_wait(time_to_wait = 5)

#Q&A의 맨 윗 사람으로 들어가기
try:
    #읽었을 때
    path = driver.find_element_by_xpath("//*[@id='listContainer_row:0']/th/a")
except:
    #안읽었을 때
    path = driver.find_element_by_xpath("//*[@id='listContainer_row:0']/th/span[1]/a")

#하이퍼링크 태그 파싱 후 진입
href = path.get_attribute('href')
driver.get(href)

#페이지 스크롤 후 기다리는 작업(잘 못긁어와서 한번 더.)
for _ in range(2):
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 끝까지 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(2)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    for i in range(0, last_height, 100):
        driver.execute_script("window.scrollTo(0, {})".format(i))
        time.sleep(0.25)
    

#html파싱
driver.implicitly_wait(time_to_wait = 20)
html = driver.page_source
soup = bs(html, 'html.parser')

#이름, 질문을 리스트로 저장
name = []
story = []
#개수 찾기 귀찮아서 그냥 except로 오류처리함
for i in range(2, 50):
    try:
        name.append(driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div[4]/div[2]/div/div[%d]/div/div/div[2]/div[1]/div[1]/div[2]/span[1]/span" %(i)).text)
    except:
        break

for i in range(1, 100):
    try:
        story.append(soup.find_all(class_ = 'vtbegenerated')[i].get_text().strip("\n"))         
    except:
        break

#데이터 프레임 변환 후 csv로 추출
df = pd.DataFrame(story, name)
speaker = input()
df.to_csv("%s" %(speaker), encoding="utf-8-sig")